In [330]:
import numpy as np
import sys
from os import path
from scipy.stats import mode
from collections import Counter

sys.path.append(path.dirname(path.dirname(path.abspath("__file__"))))
from data.dataset import FaceData
from classifier.randomforest import RandomForest

In [331]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [332]:
PATH = path.dirname(path.dirname(path.abspath("__file__"))) + "/data/face.mat"

data = FaceData(PATH)
train_X = data.train_X
train_y = data.train_y
test_X = data.test_X
test_y = data.test_y

print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)

(416, 2576) (416,)
(104, 2576) (104,)


In [392]:
num_trees=10
depth=8
split_trial=5
weak_learner="axis-align"
randomness=1

In [334]:
classifier = RandomForest(
    num_trees=num_trees,
    depth=depth,
    split_trial=split_trial,
    weak_learner=weak_learner,
    randomness=1
)

In [395]:
classifier.train(train_X, train_y)

1th tree training
2th tree training
3th tree training
4th tree training
5th tree training
6th tree training
7th tree training
8th tree training
9th tree training
10th tree training


In [380]:
for tree in classifier.trees:
    print()
    for node in tree.nodes:
        if node==None:
            print("None")
        else:
            print(node.is_leaf)
            print(Counter(node.y))


False
Counter({22: 14, 6: 13, 52: 12, 16: 12, 51: 12, 2: 11, 45: 11, 13: 11, 41: 11, 46: 11, 44: 11, 20: 10, 3: 10, 5: 10, 38: 10, 31: 10, 4: 10, 14: 9, 43: 9, 10: 9, 33: 9, 12: 9, 15: 8, 30: 8, 24: 8, 49: 8, 35: 8, 47: 8, 27: 8, 19: 7, 7: 7, 9: 7, 28: 7, 26: 7, 48: 7, 34: 7, 40: 7, 25: 7, 1: 6, 29: 6, 37: 6, 18: 6, 23: 5, 17: 5, 8: 5, 36: 5, 11: 4, 21: 4, 50: 3, 39: 3, 32: 3, 42: 2})
False
Counter({38: 9, 4: 8, 7: 7, 1: 6, 24: 6, 3: 4, 49: 3, 33: 3, 22: 3, 50: 2, 42: 2, 45: 2, 26: 2, 12: 2, 16: 1, 51: 1})
False
Counter({6: 13, 52: 12, 2: 11, 13: 11, 41: 11, 16: 11, 22: 11, 46: 11, 44: 11, 51: 11, 20: 10, 5: 10, 31: 10, 14: 9, 43: 9, 10: 9, 45: 9, 15: 8, 30: 8, 35: 8, 47: 8, 27: 8, 19: 7, 9: 7, 28: 7, 48: 7, 34: 7, 40: 7, 25: 7, 12: 7, 3: 6, 29: 6, 33: 6, 37: 6, 18: 6, 23: 5, 17: 5, 26: 5, 8: 5, 49: 5, 36: 5, 11: 4, 21: 4, 39: 3, 32: 3, 24: 2, 4: 2, 38: 1, 50: 1})
False
Counter({4: 8, 38: 6, 49: 3, 1: 2, 50: 2, 42: 2, 12: 2, 26: 1})
False
Counter({7: 7, 24: 6, 3: 4, 1: 4, 38: 3, 33: 3

In [396]:
predict = classifier.predict(test_X)
accuracy = (predict == test_y).mean()
print(accuracy)

0.16346153846153846
